In [1]:
import numpy as np
from grid import *
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib.colors as colors

In [2]:
%matplotlib qt

# General Test

In [3]:
obstacle = np.concatenate([np.arange(5,14).reshape(-1,1), np.ones(14-5).reshape(-1,1)+7], axis=1)

In [4]:
seq, board = set_board(20,20,obstacle,targets=[[2,2],[7,2]], agents=[[11,18]],beta=10)

d = np.ones(1)
while np.max(d)>0.0000001:
    d = update(board, seq)

In [5]:
value = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq))).reshape(22,22)

In [6]:
threshold=0.0001
fig, ax = plt.subplots()
im = ax.imshow(value.reshape(22,22), cmap="viridis", interpolation='none', vmax=threshold)

x,y = State.to_coord(State.targets).reshape(2,-1)
ax.plot(x+1,y+1,'x')
# plt.axis("off")
cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')

In [7]:
State.to_coord(State.agents)

array([[11, 18]])

In [8]:
# 100 step data
steps = 50
single_slice = np.array(list(map(lambda x: 0. if x is not None else 2., seq))).reshape(22,22)
dist = np.concatenate([single_slice.reshape(1,22,22),]*steps, axis=0)
start = State.to_coord(State.agents[0])
dist[0, start[0], start[1]] = 1.
for i in range(1,steps):
    for a in seq:
        if a is None:
            continue
        if State.to_token(*a.coord) in State.targets:
            cx, cy = a.coord + 1
            dist[i, cx, cy] += dist[i-1, cx, cy]
            continue
        prob = np.exp(np.array(a.Q)*State.beta)
        prob /= np.sum(prob)
        cx, cy = a.coord + 1
        for k, act in enumerate(a.action):
            x, y = act.coord + 1
            dist[i, x, y] += prob[k] * dist[i-1, cx, cy] 

In [9]:
fig, ax = plt.subplots()

threshold=1.5
im = ax.imshow(dist[0], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.5),
               vmin=0, vmax=1)

x,y = State.to_coord(State.targets).reshape(2,-1)
ax.plot(x+1,y+1,'x',color='green')
# plt.axis("off")

def animate(i):
    global dist, im
    threshold=1.5
    im = ax.imshow(dist[i], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
                   vmin=0., vmax=1.)

    x,y = State.to_coord(State.targets).reshape(2,-1)
    ax.plot(x+1,y+1,'x',color='green')
    # plt.axis("off")
    

cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')
ani = animation.FuncAnimation(fig, animate, frames=steps, interval=500) 
plt.show()

In [ ]:
ani.save('two_targets_11_18_10.gif', writer='imagemagick', fps=2)

MovieWriter imagemagick unavailable. Trying to use pillow instead.


# Two single ones

In [3]:
# from grid2 import *
import torch
import sinkhorn_torch as sk

In [14]:
obstacle = np.concatenate([np.arange(5,14).reshape(-1,1), np.ones(14-5).reshape(-1,1)+7], axis=1)

targetA = [2,2]
targetB = [7,2]
agent = [10,18]
seq, board = set_board(20,20,obstacle,targets=[targetA,], agents=[agent,],beta=3)

d = np.ones(1)
while np.max(d)>0.0000001:
    d = update(board, seq)

seq2, board2 = set_board(20,20,obstacle,targets=[targetB,], agents=[agent,],beta=3)

d = np.ones(1)
while np.max(d)>0.0000001:
    d = update(board2, seq2)

value = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq))).reshape(22,22)
value2 = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq2))).reshape(22,22)

In [15]:
threshold=0.0001
fig, ax = plt.subplots()
im = ax.imshow(value.reshape(22,22), cmap="viridis", interpolation='none', vmax=threshold)

# x,y = State.to_coord(State.targets).reshape(2,-1)
y, x = targetA
ax.plot(x+1,y+1,'x')
# plt.axis("off")
cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')

In [16]:
threshold=0.0001
fig, ax = plt.subplots()
im = ax.imshow(value2.reshape(22,22), cmap="viridis", interpolation='none', vmax=threshold)

# x,y = State.to_coord(State.targets).reshape(2,-1)
y,x = targetB
ax.plot(x+1,y+1,'x')
y,x = 9,18
ax.plot(x+1,y+1,'o')
# plt.axis("off")
cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')

In [7]:
# 100 step data
steps = 50
single_slice = np.array(list(map(lambda x: 0. if x is not None else 2., seq2))).reshape(22,22)
dist = np.concatenate([single_slice.reshape(1,22,22),]*steps, axis=0)
start = State.to_coord(State.agents[0])
dist[0, start[0], start[1]] = 1.
for i in range(1,steps):
    for a in seq2:
        if a is None:
            continue
        if State.to_token(*a.coord) in State.targets:
            cx, cy = a.coord + 1
            dist[i, cx, cy] += dist[i-1, cx, cy]
            continue
        prob = np.exp(np.array(a.Q)*State.beta)
        prob /= np.sum(prob)
        cx, cy = a.coord + 1
        for k, act in enumerate(a.action):
            x, y = act.coord + 1
            dist[i, x, y] += prob[k] * dist[i-1, cx, cy] 

In [8]:
fig, ax = plt.subplots()

threshold=1.5
im = ax.imshow(dist[0], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.5),
               vmin=0, vmax=1)

y, x = targetB
ax.plot(x+1,y+1,'x',color='green')
# plt.axis("off")

def animate(i):
    global dist, im
    threshold=1.5
    im = ax.imshow(dist[i], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
                   vmin=0., vmax=1.)

    y, x = targetB
    ax.plot(x+1,y+1,'x',color='green')
    
    y, x = targetA
    ax.plot(x+1,y+1,'x',color='green')
    # plt.axis("off")
    

cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')
ani = animation.FuncAnimation(fig, animate, frames=steps, interval=500) 
plt.show()

In [19]:
dist_normal = dist

In [20]:
# 100 step data
steps = 50
single_slice = np.array(list(map(lambda x: 0. if x is not None else 2., seq2))).reshape(22,22)
dist = np.concatenate([single_slice.reshape(1,22,22),]*steps, axis=0)
start = State.to_coord(State.agents[0])
dist[0, start[0], start[1]] = 1.
for i in range(1,steps):
    print(i,"-th step")
    for a, a_ref in zip(seq2,seq):
        if a is None:
            continue
        if State.to_token(*a.coord) in State.targets:
            cx, cy = a.coord + 1
            dist[i, cx, cy] += dist[i-1, cx, cy]
            continue
            
        rows = len(a.action)
        col_sum = torch.tensor([1., 1.],dtype=torch.float64)
        row_sum = torch.ones(int(rows),dtype=torch.float64) * 2/rows
        prob1 = np.exp(np.array(a.Q)*State.beta).reshape(-1,1)
        prob2 = np.exp(np.array(a_ref.Q)*State.beta).reshape(-1,1)
        mat = np.concatenate([prob1,prob2], axis=1)
        mat = torch.from_numpy(mat) # Do I need to copy this?
        mat = sk.sinkhorn_torch(mat, row_sum=row_sum, col_sum=col_sum)
        
        prob = mat[:,0]
        cx, cy = a.coord + 1
        if dist[i-1, cx, cy]>0.1:
            print(cx, cy, np.sum(prob1), np.sum(prob2), prob)
        for k, act in enumerate(a.action):
            x, y = act.coord + 1
            dist[i, x, y] += prob[k] * dist[i-1, cx, cy] 

1 -th step
10 18 4.3248046311838086e-33 2.5378100700724476e-29 tensor([0.1660, 0.4976, 0.1686, 0.1678], dtype=torch.float64)
2 -th step
9 18 5.720215116293558e-32 5.072899861622838e-28 tensor([0.2495, 0.2518, 0.2493, 0.2494], dtype=torch.float64)
10 17 9.016901276729599e-32 5.268246660203625e-28 tensor([0.1657, 0.4976, 0.1689, 0.1678], dtype=torch.float64)
10 19 2.0708217516370555e-34 1.2193072969747794e-30 tensor([0.1663, 0.4975, 0.1684, 0.1677], dtype=torch.float64)
11 18 5.850102703849614e-32 1.2643548232274859e-30 tensor([0.0012, 0.4987, 0.2513, 0.2487], dtype=torch.float64)
3 -th step
11 17 1.2240632654411875e-30 2.614525177549528e-29 tensor([0.0012, 0.4988, 0.2516, 0.2484], dtype=torch.float64)
11 19 2.79388507856067e-33 6.094853168249514e-32 tensor([0.0012, 0.4987, 0.2511, 0.2490], dtype=torch.float64)
12 18 1.159313776305466e-30 6.272357674398552e-32 tensor([0.0012, 0.4988, 0.2519, 0.2481], dtype=torch.float64)
4 -th step
11 18 5.850102703849614e-32 1.2643548232274859e-30 tenso

In [21]:
fig, ax = plt.subplots()

threshold=1.5
im = ax.imshow(dist[0], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.5),
               vmin=0, vmax=1)

y, x = targetB
ax.plot(x+1,y+1,'x',color='green')
# plt.axis("off")

def animate(i):
    global dist, im
    threshold=1.5 
    im = ax.imshow(dist[i], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
                   vmin=0., vmax=1.)

    y, x = targetB
    ax.plot(x+1,y+1,'x',color='green')
    
    y, x = targetA
    ax.plot(x+1,y+1,'x',color='green')
    # plt.axis("off")
    

cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')
ani = animation.FuncAnimation(fig, animate, frames=steps, interval=500) 
plt.show()

In [19]:
# 100 step data
steps = 50
single_slice = np.array(list(map(lambda x: 0. if x is not None else 2., seq2))).reshape(22,22)
dist = np.concatenate([single_slice.reshape(1,22,22),]*steps, axis=0)
start = State.to_coord(State.agents[0])
dist[0, start[0], start[1]] = 1.
for i in range(1,steps):
    print(i,"-th step")
    for a, a_ref in zip(seq2,seq):
        if a is None:
            continue
        if State.to_token(*a.coord) in State.targets:
            cx, cy = a.coord + 1
            dist[i, cx, cy] += dist[i-1, cx, cy]
            continue
            
        rows = len(a.action)
        prob1 = np.exp(np.array(a.Q)*State.beta).reshape(-1,1)
        prob2 = np.exp(np.array(a_ref.Q)*State.beta).reshape(-1,1)
        row_sum = torch.from_numpy(prob1+prob2).reshape(-1)
        row_sum /= (torch.sum(row_sum)/2)
        col_sum = torch.tensor([1., 1.],dtype=torch.float64)
        mat = np.concatenate([prob1,prob2], axis=1)
        mat = torch.from_numpy(mat) # Do I need to copy this?
        mat = sk.sinkhorn_torch(mat, row_sum=row_sum, col_sum=col_sum)
        
        prob = mat[:,0]
        cx, cy = a.coord + 1
        if dist[i-1, cx, cy]>0.1:
            print(cx, cy, np.sum(prob1), np.sum(prob2), prob)
        for k, act in enumerate(a.action):
            x, y = act.coord + 1
            dist[i, x, y] += prob[k] * dist[i-1, cx, cy] 

1 -th step
9 18 5.720215081079725e-32 5.072899861620564e-28 tensor([0.4877, 0.0012, 0.5098, 0.0012], dtype=torch.float64)
2 -th step
8 18 1.1397872772519641e-30 1.0102880145508815e-26 tensor([0.4863, 0.0012, 0.5113, 0.0012], dtype=torch.float64)
9 17 1.1911324676762728e-30 1.0567051006033464e-26 tensor([0.4891, 0.0012, 0.5085, 0.0012], dtype=torch.float64)
3 -th step
7 18 2.2637899188794683e-29 2.0060201140039027e-25 tensor([0.4852, 0.0012, 0.5124, 0.0012], dtype=torch.float64)
8 17 2.380128900884867e-29 2.110251350474632e-25 tensor([0.4872, 0.0012, 0.5104, 0.0012], dtype=torch.float64)
9 16 2.4738807860246645e-29 2.1955632528414843e-25 tensor([0.4908, 0.0012, 0.5068, 0.0012], dtype=torch.float64)
4 -th step
6 18 4.486520499997039e-28 3.974832024180991e-24 tensor([0.4846, 0.0012, 0.5129, 0.0012], dtype=torch.float64)
7 17 4.736632657408961e-28 4.1980780798519814e-24 tensor([0.4858, 0.0012, 0.5118, 0.0012], dtype=torch.float64)
8 16 4.961227990528242e-28 4.400116387902138e-24 tensor([0.

34 -th step
3 3 1.799107281294247e-07 0.009048374729046085 tensor([0.0600, 0.5182, 0.3449, 0.0769], dtype=torch.float64)
35 -th step
3 2 1.5127665725956087e-07 0.05189605504226938 tensor([0.0342, 0.0043, 0.0035, 0.9581], dtype=torch.float64)
4 3 4.161078323386055e-06 0.050128508093483695 tensor([0.9862, 0.0047, 0.0044, 0.0046], dtype=torch.float64)
36 -th step
3 3 1.799107281294247e-07 0.009048374729046085 tensor([0.0600, 0.5182, 0.3449, 0.0769], dtype=torch.float64)
37 -th step
3 2 1.5127665725956087e-07 0.05189605504226938 tensor([0.0342, 0.0043, 0.0035, 0.9581], dtype=torch.float64)
4 3 4.161078323386055e-06 0.050128508093483695 tensor([0.9862, 0.0047, 0.0044, 0.0046], dtype=torch.float64)
38 -th step
3 3 1.799107281294247e-07 0.009048374729046085 tensor([0.0600, 0.5182, 0.3449, 0.0769], dtype=torch.float64)
39 -th step
3 2 1.5127665725956087e-07 0.05189605504226938 tensor([0.0342, 0.0043, 0.0035, 0.9581], dtype=torch.float64)
4 3 4.161078323386055e-06 0.050128508093483695 tensor([0

In [20]:
fig, ax = plt.subplots()

threshold=1.5
im = ax.imshow(dist[0], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.5),
               vmin=0, vmax=1)

y, x = targetB
ax.plot(x+1,y+1,'x',color='green')
# plt.axis("off")

def animate(i):
    global dist, im
    threshold=1.5
    im = ax.imshow(dist[i], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
                   vmin=0., vmax=1.)

    y, x = targetB
    ax.plot(x+1,y+1,'x',color='green')
    
    y, x = targetA
    ax.plot(x+1,y+1,'x',color='green')
    # plt.axis("off")
    

cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')
ani = animation.FuncAnimation(fig, animate, frames=steps, interval=500) 
plt.show()

# try to avoid the other

In [4]:
agent = [9,18]
obstacle = np.concatenate([np.arange(5,14).reshape(-1,1), np.ones(14-5).reshape(-1,1)+7], axis=1)

targetA = [2,2]
targetB = [7,2]
seq, board = set_board(20,20,obstacle,targets=[targetA,], agents=[agent,],beta=3)

d = np.ones(1)
while np.max(d)>0.0000001:
    d = update(board, seq, targets=[State.to_token(targetA[0],targetA[1]).item(), State.to_token(targetB[0], targetB[1]).item()], 
               values={State.to_token(targetA[0],targetA[1]).item():0, State.to_token(targetB[0], targetB[1]).item(): -20})

seq2, board2 = set_board(20,20,obstacle,targets=[targetB,], agents=[agent,],beta=3)

d = np.ones(1)
while np.max(d)>0.0000001:
    d = update(board2, seq2,targets=[State.to_token(targetA[0],targetA[1]).item(), State.to_token(targetB[0], targetB[1]).item()], 
               values={State.to_token(targetA[0],targetA[1]).item():-20, State.to_token(targetB[0], targetB[1]).item(): 0})

value = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq))).reshape(22,22)
value2 = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq2))).reshape(22,22)

In [5]:
threshold=0.0001
fig, ax = plt.subplots()
im = ax.imshow(value.reshape(22,22), cmap="viridis", interpolation='none', vmax=threshold)

# x,y = State.to_coord(State.targets).reshape(2,-1)
y, x = targetA
ax.plot(x+1,y+1,'x')
# plt.axis("off")
cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')

In [6]:
threshold=0.0001
fig, ax = plt.subplots()
im = ax.imshow(value2.reshape(22,22), cmap="viridis", interpolation='none', vmax=threshold)

# x,y = State.to_coord(State.targets).reshape(2,-1)
y,x = targetB
ax.plot(x+1,y+1,'x')
y,x = 9,18
ax.plot(x+1,y+1,'o')
# plt.axis("off")
cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')